In [1]:
import requests
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d


In [33]:
url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_historical_population'
html = requests.get(url).content
table = pd.read_html(html)

In [34]:
population_by_state = table[3]

In [35]:
population_by_state.head()

,Name,1960,1970,1980,1990,2000,2010
0,Alabama,3266740.0,3444165,3893888.0,4040587.0,4447100.0,4779736.0
1,Alaska,226167.0,300382,401851.0,550043.0,626932.0,710231.0
2,American Samoa,20051.0,27159,32297.0,46773.0,57291.0,55519.0
3,Arizona,1302161.0,1770900,2718215.0,3665228.0,5130632.0,6392017.0
4,Arkansas,1786272.0,1923295,2286435.0,2350725.0,2673400.0,2915918.0


In [7]:
x = population_by_state.columns[1:].astype('int')
y = population_by_state.iloc[0,1:].astype('int')
f = interp1d(x, y, fill_value = 'extrapolate')

In [36]:
population_by_state['pop_func'] = [interp1d(x, y, fill_value = 'extrapolate')\
                                   for y in population_by_state[population_by_state.columns[1:]].to_numpy()]



In [19]:
f  = population_by_state[population_by_state['Name'] == 'Alaska']['pop_func']

In [24]:
f[1](1960)

array(226167.)

In [37]:
for year in range(1976,2020,4):
    population_by_state[str(year)] = [f(year) for f in population_by_state['pop_func']]

In [38]:
population_by_state.head()

,Name,1960,1970,1980,1990,2000,2010,pop_func,1976,1984,1988,1992,1996,2004,2008,2012,2016
0,Alabama,3266740.0,3444165,3893888.0,4040587.0,4447100.0,4779736.0,<scipy.interpolate.interpolate.interp1d object...,3713998.8,3952567.6,4011247.2,4121889.6,4284494.8,4580154.4,4713208.8,4846263.2,4979317.6
1,Alaska,226167.0,300382,401851.0,550043.0,626932.0,710231.0,<scipy.interpolate.interpolate.interp1d object...,361263.4,461127.8,520404.6,565420.8,596176.4,660251.6,693571.2,726890.8,760210.4
2,American Samoa,20051.0,27159,32297.0,46773.0,57291.0,55519.0,<scipy.interpolate.interpolate.interp1d object...,30241.8,38087.4,43877.8,48876.6,53083.8,56582.2,55873.4,55164.6,54455.8
3,Arizona,1302161.0,1770900,2718215.0,3665228.0,5130632.0,6392017.0,<scipy.interpolate.interpolate.interp1d object...,2339289.0,3097020.2,3475825.4,3958308.8,4544470.4,5635186.0,6139740.0,6644294.0,7148848.0
4,Arkansas,1786272.0,1923295,2286435.0,2350725.0,2673400.0,2915918.0,<scipy.interpolate.interpolate.interp1d object...,2141179.0,2312151.0,2337867.0,2415260.0,2544330.0,2770407.2,2867414.4,2964421.6,3061428.8


In [51]:
key_years = list(map(str,range(1976,2020,4)))
new_df = population_by_state.melt(id_vars = 'Name', value_vars = key_years, var_name = 'year', value_name='pop')

In [55]:
new_df.rename(columns = {'Name':'state'}, inplace = True)

In [58]:
new_df.to_csv('data/state_populations.csv',index = False)